In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import tensorflow as tf
from tensorflow.keras.layers import Dense

# --- 1. Data Loading and Preprocessing ---
d1_path = r"E:\Copy of default_of_credit_card_clients(1).csv"
d2_path = r"E:\LOAN.csv"

d1 = pd.read_csv(d1_path, header=1)
d1.rename(columns={'default payment next month': 'target'}, inplace=True)
d1 = d1.drop(columns=['ID'], errors='ignore')

d2 = pd.read_csv(d2_path)
d2.rename(columns={'y': 'target'}, inplace=True)

# Feature selection
d1_num_features = ['LIMIT_BAL', 'AGE', 'BILL_AMT1', 'PAY_AMT1']
d2_num_features = ['duration', 'age', 'campaign', 'pdays']
d1_cat_features = ['MARRIAGE']
d2_cat_features = ['marital']

overlapping_num_features = d1_num_features
overlapping_cat_features = ['MARRIAGE']

num_rename_map = {d2_col: d1_col for d2_col, d1_col in zip(d2_num_features, d1_num_features)}
cat_rename_map = {d2_col: d1_col for d2_col, d1_col in zip(d2_cat_features, d1_cat_features)}
d2_renamed = d2.rename(columns={**num_rename_map, **cat_rename_map})

all_features = overlapping_num_features + overlapping_cat_features
X1 = d1[all_features].copy()
X2 = d2_renamed[all_features].copy()
y1 = d1['target'].astype(int)
y2 = d2_renamed['target'].apply(lambda x: 1 if x == 'yes' else 0).astype(int)

# Convert categorical columns to string
for col in overlapping_cat_features:
    X1[col] = X1[col].astype(str)
    X2[col] = X2[col].astype(str)

# Clean numerical columns
for col in overlapping_num_features:
    X1[col] = pd.to_numeric(X1[col], errors='coerce')
    X1[col] = X1[col].fillna(X1[col].median())
    
    X2[col] = pd.to_numeric(X2[col], errors='coerce')
    X2[col] = X2[col].fillna(X2[col].median())

# Preprocessor is FIT on d2 (hypernetwork data)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), overlapping_num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), overlapping_cat_features)
    ])

X2_processed = preprocessor.fit_transform(X2)
X1_processed = preprocessor.transform(X1) 

# Split data
X1_train, X1_test, y1_train, y1_test = train_test_split(X1_processed, y1, test_size=0.2, random_state=42)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2_processed, y2, test_size=0.2, random_state=42)

# --- 2. Model Definition ---
INPUT_SHAPE = [X2_train.shape[1]]

def create_main_network():
    model = tf.keras.Sequential([
        Dense(64, activation='relu', input_shape=INPUT_SHAPE),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ], name="main_network")
    return model

main_network_template = create_main_network()
total_params = main_network_template.count_params()
param_shapes = [w.shape for w in main_network_template.get_weights()]
param_sizes = [tf.size(w).numpy() for w in main_network_template.get_weights()]

def create_hypernetwork(total_params):
    model = tf.keras.Sequential([
        Dense(128, activation='relu', input_shape=INPUT_SHAPE),
        Dense(total_params, activation=None)
    ], name="hypernetwork")
    return model

hypernetwork = create_hypernetwork(total_params)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.BinaryCrossentropy()

# --- 3. Hypernetwork Training on d2 ---
@tf.function
def train_step(X_batch, y_batch):
    with tf.GradientTape() as tape:
        generated_weights_flat = hypernetwork(X_batch)
        avg_weights_flat = tf.reduce_mean(generated_weights_flat, axis=0)
        
        current_pos = 0
        w1 = tf.reshape(avg_weights_flat[current_pos : current_pos+param_sizes[0]], param_shapes[0]); current_pos += param_sizes[0]
        b1 = tf.reshape(avg_weights_flat[current_pos : current_pos+param_sizes[1]], param_shapes[1]); current_pos += param_sizes[1]
        layer1_out = tf.nn.relu(tf.matmul(X_batch, w1) + b1)
        
        w2 = tf.reshape(avg_weights_flat[current_pos : current_pos+param_sizes[2]], param_shapes[2]); current_pos += param_sizes[2]
        b2 = tf.reshape(avg_weights_flat[current_pos : current_pos+param_sizes[3]], param_shapes[3]); current_pos += param_sizes[3]
        layer2_out = tf.nn.relu(tf.matmul(layer1_out, w2) + b2)

        w3 = tf.reshape(avg_weights_flat[current_pos : current_pos+param_sizes[4]], param_shapes[4]); current_pos += param_sizes[4]
        b3 = tf.reshape(avg_weights_flat[current_pos : current_pos+param_sizes[5]], param_shapes[5])
        predictions = tf.nn.sigmoid(tf.matmul(layer2_out, w3) + b3)
        
        loss = loss_fn(y_batch, predictions)

    grads = tape.gradient(loss, hypernetwork.trainable_variables)
    optimizer.apply_gradients(zip(grads, hypernetwork.trainable_variables))
    return loss

print("--- Training Hypernetwork on d2 ---")
for epoch in range(10):
    loss = train_step(tf.constant(X2_train, dtype=tf.float32), tf.constant(y2_train.values, dtype=tf.float32)[:, None])
    print(f"Epoch {epoch+1}, Hypernetwork Training Loss on d2: {loss.numpy():.4f}")

# --- 4. Evaluation on d1 (Without Fine-Tuning) ---
print("\n--- Evaluating on d1 using weights from Hypernetwork ---")
final_weights_flat = tf.reduce_mean(hypernetwork(tf.constant(X1_test, dtype=tf.float32)), axis=0)

final_weights_structured = []
current_pos = 0
for shape, size in zip(param_shapes, param_sizes):
    param = tf.reshape(final_weights_flat[current_pos : current_pos+size], shape)
    final_weights_structured.append(param)
    current_pos += size

main_network_template.set_weights(final_weights_structured)
main_network_template.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Evaluate on d1 test set
d1_accuracy = main_network_template.evaluate(X1_test, y1_test, verbose=0)[1]
print(f"\nFINAL ACCURACY on d1 test set (Direct Transfer): {d1_accuracy:.4f}")

C:\Users\Dhanushsrinivas K\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


--- Training Hypernetwork on d2 ---
Epoch 1, Hypernetwork Training Loss on d2: 0.6896
Epoch 2, Hypernetwork Training Loss on d2: 0.6740
Epoch 3, Hypernetwork Training Loss on d2: 0.6532
Epoch 4, Hypernetwork Training Loss on d2: 0.6231
Epoch 5, Hypernetwork Training Loss on d2: 0.5803
Epoch 6, Hypernetwork Training Loss on d2: 0.5228
Epoch 7, Hypernetwork Training Loss on d2: 0.4516
Epoch 8, Hypernetwork Training Loss on d2: 0.3746
Epoch 9, Hypernetwork Training Loss on d2: 0.3069
Epoch 10, Hypernetwork Training Loss on d2: 0.2673

--- Evaluating on d1 using weights from Hypernetwork ---

FINAL ACCURACY on d1 test set (Direct Transfer): 0.6800
